In [1]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn

In [2]:
def pre_processing(sentence):
    return remove_stopwords(tokenize_sentence(remove_punctuation(sentence)))

# Remove punctuation from a list of words
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', sentence)

# Remove stopwords from a list of words
def remove_stopwords(words_list):
    stopwords = open("stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords:
        stopwords_list.append(word.replace('\n', ''))
    stopwords.close()
    return [value.lower() for value in words_list if value.lower() not in stopwords_list]

# Tokenize the input sentence and also lemmatize its words
def tokenize_sentence(sentence):
    words_list = []
    lmtzr = WordNetLemmatizer()
    for tag in nltk.pos_tag(word_tokenize(sentence)):
        if (tag[1][:2] == "NN"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.NOUN))
        elif (tag[1][:2] == "VB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.VERB))
        elif (tag[1][:2] == "RB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADV))
        elif (tag[1][:2] == "JJ"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADJ))
    return words_list

In [ ]:
# Topic modelling using gensim
from gensim import corpora, models

FILE = 'italian_cuisine_2.txt'  # il file contiene un insieme di documenti divisi per paragrafi
TOPIC_NUMBER = 10
WORDS_FOR_TOPIC = 5

with open(FILE, encoding='utf-8') as file:
    # for each doc create list of pre-processed words in that doc (list of lists)
    list_document_words = []

    for line in file:
        if "<doc" in line:  # tag for new doc
            document_words = [] 
            while True:
                next_line = file.readline()

                # remove unuseful tags
                new_next_line = next_line.replace("<p> ", "").replace("</p>\n", "").replace("/p", "")

                if "</doc>" in next_line:
                    break
                # pre-processing steps
                sentence_words = pre_processing(new_next_line)
                document_words.extend(sentence_words)
            list_document_words.append(document_words)
    file.close()

    print(len(list_document_words), " documents")

    # Create a dict with integer keys for all words
    dictionary_LDA = corpora.Dictionary(list_document_words)

    
    # elimina tutti i termini che NON appaiono in almeno 3 documenti
    # elimina tutti i termini che NON appaiono in oltre il 50% dei documenti (vedere doc. ufficiale di filter_extremes)
    dictionary_LDA.filter_extremes(no_below=3)

    # Converts each document into a list of BoW (list of (id_term, term_frequency) for each term in doc)
    corpus = [dictionary_LDA.doc2bow(document_words) for document_words in list_document_words]

    lda_model = models.LdaModel(corpus, num_topics=TOPIC_NUMBER, \
                                id2word=dictionary_LDA, \
                                passes=4, alpha=[0.01] * TOPIC_NUMBER, \
                                eta=[0.01] * len(dictionary_LDA.keys()))

    for i, topic in lda_model.show_topics(formatted=True, num_topics=TOPIC_NUMBER, num_words=WORDS_FOR_TOPIC):
        print(str(i) + ": " + topic)
        print()

    # distribuzione di probabilità dei topic nel documento 5
    print(lda_model[corpus[5]])  # corpus[0] means the first document.

310  documents
